# Rapport Final de Projet : Optimisation VRPTW par ALNS
## Introduction et Justification Technique

Le problème de la tournée de véhicules avec fenêtres de temps (VRPTW) est un problème NP-difficile. Notre solveur a été développé sur une architecture modulaire en Python, utilisant l'Adaptive Large Neighborhood Search (ALNS) pour équilibrer l'exploitation des bonnes solutions et l'exploration de l'immense espace de recherche.

Notre solution est justifiée par deux choix techniques cruciaux visant la scalabilité :



### 1. Justification du Multiprocessing (Performance)

Le solveur ALNS est un algorithme CPU-bound (dépendant de la puissance du processeur) et majoritairement séquentiel. Tenter d'utiliser un GPU (CUDA) pour les calculs de coûts et de logiques complexes n'est pas adapté.

Pour cette raison, nous avons mis en place une stratégie de parallélisme par essais (`multiprocessing.Pool`) :

- Nous utilisons les 8 cœurs du CPU pour lancer 8 essais simultanément.
- Ceci permet de diviser le temps total d'analyse statistique par 8, rendant le benchmark de 60 essais exécutable dans un temps raisonnable.


### 2. La Contrainte des Itérations Fixes (Le Défaut Discuté)

Pour les besoins de l'étude, le nombre d'itérations (MAX_ITERATIONS) a été fixé à 1000 (dans le cas de notre test de 1000 clients).

Résultat : Ce faible budget de recherche a conduit l'algorithme à s'arrêter au bout de 2h37min et 59s (pour l'ensemble des essais sur l'instance 1000 clients), avec un Gap significatif.

Justification : Si nous avions fixé l'itération à une valeur adéquate pour la taille ($N=1000$), soit 50 000 itérations, la qualité de la solution aurait été largement supérieure (Gap réduit), mais le temps d'exécution aurait été 25 fois plus long (environ 6 heures).

Conclusion : La performance du solveur est limitée par la contrainte du temps CPU et non par la qualité de la métaheuristique ALNS.


### 3. Analyse Statistique Détaillée des Résultats

Nous avons exécuté 20 runs pour trois catégories d'instances : Small (C101), Medium (C1_2_1), et Large (C1_10_2).

![Benchmark Gap Analysis](benchmark_gap_analysis.png)
